In [1]:
import pandas as pd
import numpy as np
import pickle
from time import ctime
import plotly.express as px
import plotly.graph_objs as go
from mat.deloitte.ansfinland.anpo_sectorisation_poc.backend.utils import *
from mat.deloitte.ansfinland.anpo_sectorisation_poc.backend.taskload_model import *

import pyomo.environ as pyo
from pyomo.opt import SolverFactory
from pyomo.core.base import Constraint as pyo_constraint
from pyomo.core.base import Var as pyo_vars

pd.options.display.float_format = '{:.5f}'.format

## Utility functions

Create a function that takes a tfv and returns all the configurations it features in.

In [2]:
df_configuration_definition = load_configuration_definition()
df_configuration_definition


,Configuration,TFVs,Sectors
0,CONF1,[EFINALL],"[[EFINA, EFINB, EFINC, EFIND, EFINE, EFINF, EF..."
1,CONF2,"[EF4ABCD, EF10EN]","[[EFINA, EFINB, EFINC, EFIND], [EFINH, EFINJ, ..."
2,CONF2A,"[EF6AF, EF8GN]","[[EFINA, EFINB, EFINC, EFIND, EFINE, EFINF], [..."
3,CONF2B,"[EF7AG, EF7VN]","[[EFINA, EFINB, EFINC, EFIND, EFINE, EFINF, EF..."
4,CONF2C,"[EF12XHJ, EF2HJ]","[[EFINA, EFINB, EFINC, EFIND, EFINE, EFINF, EF..."
5,CONF2D,"[EF10AFKN, EF4GVHJ]","[[EFINA, EFINB, EFINC, EFIND, EFINE, EFINF, EF..."
6,CONF2E,"[EF6ADLN, EF8EM]","[[EFINA, EFINB, EFINC, EFIND, EFINL, EFINN], [..."
7,CONF2F,"[EF10AJ, EF4KLMN]","[[EFINA, EFINB, EFINC, EFIND, EFINE, EFINF, EF..."
8,CONF2G,"[EF11XHJV, EF3VHJ]","[[EFINA, EFINB, EFINC, EFIND, EFINE, EFINF, EF..."
9,CONF2H,"[EF11XDEF, EF3DEF]","[[EFINA, EFINB, EFINC, EFING, EFINV, EFINH, EF..."


In [3]:
def get_config_tfvs_list(df_configuration_definition, config):
    
    return df_configuration_definition[df_configuration_definition['Configuration'] == config]['TFVs'].values[0]

In [45]:
get_config_tfvs_list(df_configuration_definition, 'CONF1')

['EFINALL']

In [4]:
tfv_list = list(set([item for sublist in df_configuration_definition.TFVs.to_list() for item in sublist]))
tfv_list

['EF6GVKN',
 'EF9AV',
 'EFINE',
 'EF2HJ',
 'EF8ADKN',
 'EF4EFGV',
 'EF3EFG',
 'EF3VKM',
 'EF7MID',
 'EF10ACHN',
 'EF4DEFG',
 'EF10EN',
 'EF11XHJV',
 'EF4HJNV',
 'EF8EGVN',
 'EF12XHJ',
 'EF2FG',
 'EF11XDEF',
 'EF4ABCD',
 'EF4KLMN',
 'EF4GVHJ',
 'EF5VHJKM',
 'EF6ADLN',
 'EF10AGKM',
 'EF2EF',
 'EF3ABC',
 'EF6GM',
 'EF6AF',
 'EF5KLMNV',
 'EF3VHJ',
 'EF10AJ',
 'EF3DEF',
 'EF5GKLMN',
 'EF4GVKM',
 'EF7VN',
 'EFINALL',
 'EF8EM',
 'EF10AFKN',
 'EF8GN',
 'EF7AG',
 'EF6EJ',
 'EF2KM',
 'EF2GV',
 'EF2LN']

In [5]:
data_list = []

for tfv in tfv_list:
    mask = df_configuration_definition.TFVs.apply(lambda x: True if tfv in x else False)
    temp_df = df_configuration_definition[mask]
    configs = temp_df.Configuration.to_list()
    
    data_list.append([tfv, configs])

columns = ['Traffic_Flow_Volume', 'Configurations']
df_tfv_configs = pd.DataFrame(data_list, columns=columns)
df_tfv_configs

,Traffic_Flow_Volume,Configurations
0,EF6GVKN,"[CONF3L, CONF4D]"
1,EF3ABC,"[CONF3J, CONF3P, CONF4J, CONF4K, CONF5G, CONF5H]"
2,EF3VKM,"[CONF4L, CONF5C]"
3,EF6AF,"[CONF2A, CONF3B, CONF3C, CONF3E, CONF3L, CONF4C]"
4,EF2LN,"[CONF3E, CONF3I, CONF3K, CONF3O, CONF4C, CONF4..."
5,EF3EFG,"[CONF3A, CONF4F, CONF4H, CONF4N, CONF5, CONF5C]"
6,EF6GM,"[CONF3E, CONF4O]"
7,EF9AV,[CONF3N]
8,EF5GKLMN,"[CONF3B, CONF4, CONF4K]"
9,EF10ACHN,[CONF2I]


In [5]:
df_tfv_configs = load_tfv_configs()

In [45]:
#df_tfv_configs.to_pickle('configs_containing_tfv.pickle')

In [6]:
num_airspaces_in_config_dict = {config: [int(i) for i in config if i.isdigit()][0] for config in df_configuration_definition.Configuration.to_list()}

In [8]:
num_airspaces_in_config_dict

{'CONF1': 1,
 'CONF2': 2,
 'CONF2A': 2,
 'CONF2B': 2,
 'CONF2C': 2,
 'CONF2D': 2,
 'CONF2E': 2,
 'CONF2F': 2,
 'CONF2G': 2,
 'CONF2H': 2,
 'CONF2I': 2,
 'CONF2J': 2,
 'CONF2L': 2,
 'CONF3': 3,
 'CONF3A': 3,
 'CONF3B': 3,
 'CONF3C': 3,
 'CONF3D': 3,
 'CONF3E': 3,
 'CONF3F': 3,
 'CONF3G': 3,
 'CONF3H': 3,
 'CONF3I': 3,
 'CONF3J': 3,
 'CONF3K': 3,
 'CONF3L': 3,
 'CONF3M': 3,
 'CONF3N': 3,
 'CONF3O': 3,
 'CONF3P': 3,
 'CONF4': 4,
 'CONF4B': 4,
 'CONF4C': 4,
 'CONF4D': 4,
 'CONF4E': 4,
 'CONF4F': 4,
 'CONF4G': 4,
 'CONF4H': 4,
 'CONF4I': 4,
 'CONF4J': 4,
 'CONF4K': 4,
 'CONF4L': 4,
 'CONF4M': 4,
 'CONF4N': 4,
 'CONF4O': 4,
 'CONF5': 5,
 'CONF5A': 5,
 'CONF5B': 5,
 'CONF5C': 5,
 'CONF5D': 5,
 'CONF5E': 5,
 'CONF5F': 5,
 'CONF5G': 5,
 'CONF5H': 5}

## Get taskload data

Run the taskload model to get data, then process into desired format.

In [7]:
elementry_sector_list = load_elementry_sector_list()
df_traffic = load_schedule_data()
df_traffic = subset_traffic_dataframe_by_airspaces(df_traffic, elementry_sector_list)
time_horizon_start = mins_after_midnight_to_timestamp(0)
time_horizon_end = mins_after_midnight_to_timestamp(1440)
df_tfv = load_tfv_definition()

In [8]:
time_interval_length_mins = 15


df_sector_taskloads = run_taskload_model_elementry_sectors(df_traffic, time_horizon_start, time_horizon_end,
                                                           time_interval_length_mins=time_interval_length_mins)

In [11]:
df_sector_taskloads

,elementry_sector,time_interval_start,time_interval_end,Exit,Sequencing,Altitude Change,Monitoring,Entry,total_taskload
0,EFINA,2019-08-29 00:00:00+03:00,2019-08-29 00:15:00+03:00,0.000000,0.0,0.0,0.000000,0.000000,0.000000
1,EFINB,2019-08-29 00:00:00+03:00,2019-08-29 00:15:00+03:00,0.000000,0.0,0.0,0.000000,0.000000,0.000000
2,EFINC,2019-08-29 00:00:00+03:00,2019-08-29 00:15:00+03:00,0.000000,0.0,0.0,0.000000,0.000000,0.000000
3,EFIND,2019-08-29 00:00:00+03:00,2019-08-29 00:15:00+03:00,0.000000,0.0,0.0,0.000000,0.000000,0.000000
4,EFINE,2019-08-29 00:00:00+03:00,2019-08-29 00:15:00+03:00,0.000000,0.0,0.0,0.000000,0.000000,0.000000
5,EFINF,2019-08-29 00:00:00+03:00,2019-08-29 00:15:00+03:00,0.000000,0.0,0.0,0.000000,0.000000,0.000000
6,EFING,2019-08-29 00:00:00+03:00,2019-08-29 00:15:00+03:00,0.000000,0.0,0.0,0.000130,0.019333,0.019463
7,EFINH,2019-08-29 00:00:00+03:00,2019-08-29 00:15:00+03:00,0.000000,0.0,0.0,0.000000,0.000000,0.000000
8,EFINJ,2019-08-29 00:00:00+03:00,2019-08-29 00:15:00+03:00,0.000000,0.0,0.0,0.000000,0.000000,0.000000
9,EFINK,2019-08-29 00:00:00+03:00,2019-08-29 00:15:00+03:00,0.000000,0.0,0.0,0.000000,0.000000,0.000000


In [9]:
def get_taskloads_by_airspace_df(df_sector_taskloads, tfv_list, df_tfv=None):
    
    if df_tfv is None:
        df_tfv = load_tfv_definition()
        
    out_df = pd.concat([get_airspace_taskloads_per_time_interval(df_sector_taskloads, df_tfv, tfv) 
                        for tfv in tfv_list],ignore_index=True)
    
    return out_df

In [10]:
df_airspace_taskloads = get_taskloads_by_airspace_df(df_sector_taskloads, tfv_list, df_tfv=None)

Write a function to get the airspace taskload data into the correct format for Pyomo to use.

In [11]:
def airspace_taskload_df_to_dict(df_airspace_taskloads):
    
    out_dict = {}
    
    airspace_id_list = list(set(df_airspace_taskloads.airspace_id.to_list()))
    time_interval_start_list = sorted(list(set(df_airspace_taskloads.time_interval_start.to_list())))
    
    for tfv in airspace_id_list:
        for counter, time_interval in enumerate(time_interval_start_list):
            taskload = df_airspace_taskloads[(df_airspace_taskloads['airspace_id'] == tfv)
                                            &(df_airspace_taskloads['time_interval_start'] == time_interval)
                                            ]['total_taskload'].values[0]
            out_dict[(tfv, counter)] = taskload
            
    return out_dict

In [12]:
airspace_taskload_dict = airspace_taskload_df_to_dict(df_airspace_taskloads)

Write a function to get the configs that contain each airspace into the correct format for Pyomo to use.

In [13]:
def get_tfv_configs_list(df_tfv_configs, tfv):
    
    return df_tfv_configs[df_tfv_configs['Traffic_Flow_Volume'] == tfv]['Configurations'].values[0]

def airspace_to_configs_df_to_dict(df_tfv_configs):
    
    out_dict = {}
    
    for tfv in df_tfv_configs.Traffic_Flow_Volume.to_list():
        out_dict[tfv] = get_tfv_configs_list(df_tfv_configs, tfv)

    return out_dict

In [14]:
airspace_to_configs_dict = airspace_to_configs_df_to_dict(df_tfv_configs)

## Build model

This section will contain the mathematical model + Pyomo code for the model.

__Sets__

$T$: Set of time intervals

$A$: Set of airspaces

$C$: Set of airspace configurations

$C_{a}$: Set of configurations in which airspace $a$ is active

__Decision Variables__

$m_{a,t} = \left\{ \begin{matrix}1 \text{ if airspace } a \text{ is active in time interval } t \\ 0 \text{ otherwise} \end{matrix} \right.$

$n_{a,t} = \left\{ \begin{matrix}1 \text{ if airspace } a \text{ is active in time interval } t \text{ and needs two controllers to manage it}\\ 0 \text{ otherwise} \end{matrix} \right.$

$p_{c,t} = \left\{ \begin{matrix}1 \text{ if configuration } c \text{ is active in time interval } t \\ 0 \text{ otherwise} \end{matrix} \right.$

$q_{t} = \text{number of airspaces active in time interval t}$

$r_{c,t} = \left\{ \begin{matrix}1 \text{ if configuration } c \text{ switches on or off between in time interval } t \text{ and } t + 1\\ 0 \text{ otherwise} \end{matrix} \right.$

$s_{t} = \left\{ \begin{matrix}1 \text{ if any configuration switches on or off between in time interval } t \text{ and } t + 1\\ 0 \text{ otherwise} \end{matrix} \right.$

__Model Data__

$W_{a,t}$: The taskload for airspace $a$ in time interval $t$

__Model Parameters__

$\gamma_{1}$: Weight factor for minimising number of airspaces requiring two controllers

$\gamma_{2}$: Weight factor for minimising number of configuration switches

$\epsilon$: Tiny number used for constraint building - default to $10^{-5}$

$L_{i}$: The taskload limit for an airspace with $i \in \{1,2\}$ controllers

$K$: The minimum number of time intervals allowed between airspace reconfigurations

__Objective Function__

The first term minimises the number of airspaces open over the time horizon. The second term mimises the number of those airspaces that require both an executive and planning controller. The third term minimises the number of airspace configuration changes.

$$ \min \left( \sum_{t \in \mathcal{T}} q_{t} + \gamma_{1} \sum_{a \in \mathcal{A}} \sum_{t \in \mathcal{T}} n_{at} + \gamma_{2} \sum_{t=0}^{\vert\mathcal{T}\vert-1} s_{t} \right) $$

__Constraints__

This constraint set links together the $m$ and $p$ variables:

$$ m_{a,t} = \sum_{c \in \mathcal{C_{a}}} p_{c,t} \hspace{0.5cm} \forall a \in \mathcal{A}, \forall t \in \mathcal{T}$$

This constraint set links together the $m$ and $q$ variables:

$$ q_{t} = \sum_{a \in \mathcal{A}} m_{a,t} \hspace{0.5cm} \forall t \in \mathcal{T}$$

These constraint sets link together the $m$ and $n$ variables:

$$ (\epsilon + L_{1}) \hspace{0.05cm} n_{a,t}  \leq W_{a,t} \hspace{0.05cm} m_{a,t} \hspace{0.5cm} \forall a \in \mathcal{A},  \forall t \in \mathcal{T}$$

$$ (W_{a,t} - L_{1})\hspace{0.05cm} \epsilon \hspace{0.05cm} m_{a,t}  \leq n_{a,t} \hspace{0.5cm} \forall a \in \mathcal{A}, \forall t \in \mathcal{T}$$

These constraint sets link together the $c$ and $r$ variables:

$$ r_{c,t} \geq p_{c,t} - p_{c,t+1} \hspace{0.5cm} \forall c \in \mathcal{C}, \forall t \in \left[0, \vert \mathcal{T} \vert -1 \right] $$

$$ r_{c,t} \geq p_{c,t+1} - p_{c,t} \hspace{0.5cm} \forall c \in \mathcal{C}, \forall t \in \left[0, \vert \mathcal{T} \vert -1 \right] $$

$$ r_{c,t} \leq p_{c,t} + p_{c,t+1} \hspace{0.5cm} \forall c \in \mathcal{C}, \forall t \in \left[0, \vert \mathcal{T} \vert -1 \right] $$

$$ r_{c,t} \leq 2 - p_{c,t} - p_{c,t+1} \hspace{0.5cm} \forall c \in \mathcal{C}, \forall t \in \left[0, \vert \mathcal{T} \vert -1 \right] $$

This constraint set links together the $r$ and $s$ variables:

$$ \sum_{c \in \mathcal{C}} r_{c,t} = 2 s_{t} \hspace{0.5cm} \forall t \in \mathcal{T} $$

This constraint set ensures that one and only one airspace configuration is active in every time interval:

$$ \sum_{c \in \mathcal{C}} p_{c,t} = 1 \hspace{0.5cm} \forall t \in \mathcal{T} $$

This constraint set ensures that no airspace is active if its taskload is higher than the limit for two controllers:

$$ W_{a,t} \hspace{0.05cm} m_{a,t} \leq L_{2} \hspace{0.5cm} \forall a \in \mathcal{A}, \forall t \in \mathcal{T} $$

This constraint set imposes a limit on the minimum number of allowed time intervals between airspace configuration changes:

$$ \sum_{i=t-K+1}^{t+K-1} s_{i} \leq 2 - s_{t} \hspace{0.5cm} \forall t \in \mathcal{T} $$

In [15]:
def objective_function_1(model):
    
    return sum(model.q[t] for t in model.T)

def objective_function_2(model):
    
    return sum(model.q[t] for t in model.T) + model.weight_1 * sum(sum(model.n[a, t] for a in model.A) for t in model.T)

def objective_function_3(model):
    
    return sum(model.q[t] for t in model.T) + model.weight_1 * sum(sum(model.n[a, t] for a in model.A) for t in model.T) +\
           model.weight_2 * sum(model.s[t] for t in model.T_minus)

In [16]:
def constraint_link_m_and_p(model, a, t):
    
    return model.m[a, t] == sum(model.p[c, t] for c in model.configs_containing_airspace[a])


def constraint_link_m_and_q(model, t):
    
    ## could this be changed to take some data on the # sectors each config has - might be better?
    
    return model.q[t] == sum(model.m[a, t] for a in model.A)


def constraint_link_m_and_n_1(model, a, t):
    
    return (model.epsilon + model.single_controller_taskload_limit) * model.n[a, t] <= model.taskload[a, t] * model.m[a, t]

def constraint_link_m_and_n_2(model, a, t):

    # model won't compile if taskload isn't made into a float, no idea why - doesn't affect other constraints
    delta = ((float(model.taskload[a, t]) - model.single_controller_taskload_limit) * model.epsilon)
    
    return delta * model.m[a, t] <= model.n[a, t] 

def constraint_full_coverage(model, t):
    
    return sum(model.p[c, t] for c in model.C) == 1

def constraint_task_limit(model, a, t):
    
    return model.taskload[a, t] * model.m[a, t] <= model.double_controller_taskload_limit


#def constraint_link_c_and_r_1(model, t):
#    
#    sum_t = sum(model.p[c, t] * model.config_index_dict[c] for c in model.C)
#    sum_t_plus_1 = sum(model.p[c, t+1] * model.config_index_dict[c] for c in model.C)
#    
#    return sum_t - sum_t_plus_1 <= model.config_max_index * model.r[t]
#
#def constraint_link_c_and_r_2(model, t):
#    
#    sum_t = sum(model.p[c, t] * model.config_index_dict[c] for c in model.C)
#    sum_t_plus_1 = sum(model.p[c, t+1] * model.config_index_dict[c] for c in model.C)
#    
#    return sum_t_plus_1 - sum_t <= model.config_max_index * model.r[t]


def constraint_link_c_and_r_3(model, c, t):
    
    return model.r[c, t] >= model.p[c, t+1] - model.p[c, t]

def constraint_link_c_and_r_4(model, c, t):
    
    return model.r[c, t] >= model.p[c, t] - model.p[c, t+1]

def constraint_link_c_and_r_5(model, c, t):
    
    return model.r[c, t] <= model.p[c, t+1] + model.p[c, t]

def constraint_link_c_and_r_6(model, c, t):
    
    return model.r[c, t] <= 2 - model.p[c, t+1] - model.p[c, t]

def constraint_link_r_and_s_1(model, t):
    
    return sum(model.r[c, t] for c in model.C)/2 == model.s[t]

#def constraint_link_r_and_s_2(model, t):
    
#    return sum(model.r[c, t] for c in model.C) >= model.s[t]

def constraint_min_reconfig_period(model, t):
    
    return sum(model.s[t2] for t2 in range(t-model.min_reconfig_period+1, t+model.min_reconfig_period)) <= (2 - model.s[t])



#def constraint_min_reconfig_period(model, t):
#    
#    return model.min_reconfig_period * (model.r[t] + (1 - model.r[t-1])) >=\
#           sum(model.r[t2] for t2 in range(t+1, t+model.min_reconfig_period))


#def constraint_min_reconfig_period(model, t):
    
#    return sum(model.r[t2] for t2 in range(t-model.min_reconfig_period, t+model.min_reconfig_period+1)) <= 1

In [20]:
def build_model(parameters, df_tfv_configs, df_configuration_definition, df_airspace_taskloads, 
                airspace_taskload_dict, airspace_to_configs_dict, weight_1):
    
    double_controller_taskload_limit = parameters['double_controller_taskload_limit']
    single_controller_taskload_limit = parameters['single_controller_taskload_limit']
    number_time_intervals = parameters['number_time_intervals']
    epsilon = parameters['epsilon']
    config_index_dict = {config: counter for counter, config 
                         in enumerate(df_configuration_definition.Configuration.to_list())}
    
    # define model
    model = pyo.ConcreteModel()
    
    # define parameters
    model.double_controller_taskload_limit = pyo.Param(initialize=double_controller_taskload_limit, 
                                                       within=pyo.NonNegativeReals)
    model.single_controller_taskload_limit = pyo.Param(initialize=single_controller_taskload_limit, 
                                                       within=pyo.NonNegativeReals)
    model.number_time_intervals = pyo.Param(initialize=number_time_intervals, 
                                            within=pyo.NonNegativeReals)
    model.epsilon = pyo.Param(initialize=epsilon, 
                              within=pyo.NonNegativeReals)
    model.weight_1 = pyo.Param(initialize=weight_1, 
                               within=pyo.NonNegativeReals)
    
    # define model sets
    model.T = pyo.RangeSet(0, model.number_time_intervals-1, 1, dimen=1, ordered=True)
    
    #model.T_minus = pyo.RangeSet(0, model.number_time_intervals-2, 1, dimen=1, ordered=True)
    
    
    model.A = pyo.Set(initialize=list(set(df_airspace_taskloads.airspace_id.to_list())), dimen=1)
    model.C = pyo.Set(initialize=df_configuration_definition.Configuration.to_list(), dimen=1)
    
    # add data to model
    model.taskload = pyo.Param(model.A, model.T, within=pyo.NonNegativeReals, 
                              initialize=airspace_taskload_dict, default=0.0)
    model.configs_containing_airspace = pyo.Param(model.A, initialize=airspace_to_configs_dict)
    
    #model.config_index_dict = pyo.Param(model.C, initialize=config_index_dict)
    
    
    # define varibles
    model.m = pyo.Var(model.A, model.T, within=pyo.Binary)
    model.n = pyo.Var(model.A, model.T, within=pyo.Binary)
    model.p = pyo.Var(model.C, model.T, within=pyo.Binary)
    model.q = pyo.Var(model.T, within=pyo.PositiveIntegers)
    
    #model.r = pyo.Var(model.T_minus, within=pyo.Binary)
    
    # define objective value
    #model.objective_function_1 = pyo.Objective(rule=objective_function_1, sense=pyo.minimize)
    model.objective_function_2 = pyo.Objective(rule=objective_function_2, sense=pyo.minimize)
    
    #model.objective_function_3 = pyo.Objective(rule=objective_function_3, sense=pyo.minimize)
    
    # define constraints
    model.constraint_link_m_and_p = pyo.Constraint(model.A, model.T, rule=constraint_link_m_and_p)
    model.constraint_link_m_and_q = pyo.Constraint(model.A, model.T, rule=constraint_link_m_and_q)
    model.constraint_link_m_and_n_1 = pyo.Constraint(model.A, model.T, rule=constraint_link_m_and_n_1)
    model.constraint_link_m_and_n_2 = pyo.Constraint(model.A, model.T, rule=constraint_link_m_and_n_2)
    model.constraint_full_coverage = pyo.Constraint(model.T, rule=constraint_full_coverage)
    model.constraint_task_limit = pyo.Constraint(model.A, model.T, rule=constraint_task_limit)
    
    #model.constraint_link_c_and_r_1 = pyo.Constraint(model.T_minus, rule=constraint_link_c_and_r_1)
    #model.constraint_link_c_and_r_2 = pyo.Constraint(model.T_minus, rule=constraint_link_c_and_r_2)
    
    return model

In [17]:
def build_model(parameters, df_tfv_configs, df_configuration_definition, df_airspace_taskloads, 
                airspace_taskload_dict, airspace_to_configs_dict):
    
    double_controller_taskload_limit = parameters['double_controller_taskload_limit']
    single_controller_taskload_limit = parameters['single_controller_taskload_limit']
    number_time_intervals = parameters['number_time_intervals']
    epsilon = parameters['epsilon']
    weight_1 = parameters['weight_1']
    weight_2 = parameters['weight_2']
    
    min_reconfig_period = parameters['min_reconfig_period']
    
    config_index_dict = {config: counter for counter, config 
                         in enumerate(df_configuration_definition.Configuration.to_list())}
    
    # define model
    model = pyo.ConcreteModel()
    
    # define parameters
    model.double_controller_taskload_limit = pyo.Param(initialize=double_controller_taskload_limit, 
                                                       within=pyo.NonNegativeReals)
    model.single_controller_taskload_limit = pyo.Param(initialize=single_controller_taskload_limit, 
                                                       within=pyo.NonNegativeReals)
    model.number_time_intervals = pyo.Param(initialize=number_time_intervals, 
                                            within=pyo.PositiveIntegers)
    model.epsilon = pyo.Param(initialize=epsilon, 
                              within=pyo.NonNegativeReals)
    
    model.min_reconfig_period = pyo.Param(initialize=min_reconfig_period, 
                                          within=pyo.PositiveIntegers)
    
    model.weight_1 = pyo.Param(initialize=weight_1, 
                               within=pyo.NonNegativeReals)
    model.weight_2 = pyo.Param(initialize=weight_2, 
                               within=pyo.NonNegativeReals)
    model.config_max_index = pyo.Param(initialize=max(config_index_dict.values()), 
                                       within=pyo.PositiveIntegers)
    
    # define model sets
    model.T = pyo.RangeSet(0, model.number_time_intervals-1, 1, dimen=1, ordered=True)
    model.T_minus = pyo.RangeSet(0, model.number_time_intervals-2, 1, dimen=1, ordered=True)

    model.T_min_reconfig_period = pyo.RangeSet(min_reconfig_period, model.number_time_intervals-min_reconfig_period-2,
                                               1, dimen=1, ordered=True)
    
    model.A = pyo.Set(initialize=list(set(df_airspace_taskloads.airspace_id.to_list())), dimen=1)
    model.C = pyo.Set(initialize=df_configuration_definition.Configuration.to_list(), dimen=1)
    
    # add data to model
    model.taskload = pyo.Param(model.A, model.T, within=pyo.NonNegativeReals, 
                              initialize=airspace_taskload_dict, default=0.0)
    model.configs_containing_airspace = pyo.Param(model.A, initialize=airspace_to_configs_dict)
    model.config_index_dict = pyo.Param(model.C, initialize=config_index_dict)
    
    
    # define varibles
    model.m = pyo.Var(model.A, model.T, within=pyo.Binary)
    model.n = pyo.Var(model.A, model.T, within=pyo.Binary)
    model.p = pyo.Var(model.C, model.T, within=pyo.Binary)
    model.q = pyo.Var(model.T, within=pyo.PositiveIntegers)
    
    #model.r = pyo.Var(model.T_minus, within=pyo.Binary)
    
    model.r = pyo.Var(model.C, model.T_minus, within=pyo.Binary)
    model.s = pyo.Var(model.T_minus, within=pyo.Binary)
    
    # define objective value
    #model.objective_function_1 = pyo.Objective(rule=objective_function_1, sense=pyo.minimize)
    #model.objective_function_2 = pyo.Objective(rule=objective_function_2, sense=pyo.minimize)
    model.objective_function_3 = pyo.Objective(rule=objective_function_3, sense=pyo.minimize)
    
    # define constraints
    model.constraint_link_m_and_p = pyo.Constraint(model.A, model.T, rule=constraint_link_m_and_p)
    model.constraint_link_m_and_q = pyo.Constraint(model.T, rule=constraint_link_m_and_q)
    model.constraint_link_m_and_n_1 = pyo.Constraint(model.A, model.T, rule=constraint_link_m_and_n_1)
    model.constraint_link_m_and_n_2 = pyo.Constraint(model.A, model.T, rule=constraint_link_m_and_n_2)
    model.constraint_full_coverage = pyo.Constraint(model.T, rule=constraint_full_coverage)
    model.constraint_task_limit = pyo.Constraint(model.A, model.T, rule=constraint_task_limit)
    
    #model.constraint_link_c_and_r_1 = pyo.Constraint(model.T_minus, rule=constraint_link_c_and_r_1)
    #model.constraint_link_c_and_r_2 = pyo.Constraint(model.T_minus, rule=constraint_link_c_and_r_2)
    
    model.constraint_link_c_and_r_3 = pyo.Constraint(model.C, model.T_minus, rule=constraint_link_c_and_r_3)
    model.constraint_link_c_and_r_4 = pyo.Constraint(model.C, model.T_minus, rule=constraint_link_c_and_r_4)
    model.constraint_link_c_and_r_5 = pyo.Constraint(model.C, model.T_minus, rule=constraint_link_c_and_r_5)
    model.constraint_link_c_and_r_6 = pyo.Constraint(model.C, model.T_minus, rule=constraint_link_c_and_r_6)
    model.constraint_link_r_and_s_1 = pyo.Constraint(model.T_minus, rule=constraint_link_r_and_s_1)
    
    model.constraint_min_reconfig_period = pyo.Constraint(model.T_min_reconfig_period, 
                                                          rule=constraint_min_reconfig_period)
    
    
    return model

In [18]:
def solve_model(model, solver_name, solver_path, timeout_time=120, ratio_gap=0.01, show_working=True):
    
    print('Run start time: ' + str(ctime()))
    
    # Use CPLEX on the NEOS server
    if solver_name == 'cplex':
        manager = pyo.SolverManagerFactory('neos')
        opt_settings = SolverFactory(solver_name)
        opt_settings.set_options('mipgap=' + str(ratio_gap))
        opt_settings.set_options('timelimit=' + str(timeout_time))
        opt_settings.set_options('mipdisplay=' + str(3))
        opt_settings.set_options('nodefile=' + str(2))
        opt_settings.set_options('treememory=' + str(10000))
        results = manager.solve(model, opt=opt_settings, keepfiles=True)
        
    elif solver_name == 'cbc':
        opt_settings = SolverFactory(solver, executable=solver_filepath)
        opt_settings.set_options('sec=' + str(max_run_time))
        opt_settings.set_options('ratioGap=' + str(ratio_gap))
        results = opt.solve(model, tee=show_working)
        
    else:
        raise ValueError(f'Solver {solver_name} not supported')
        
    print('Run finish time: ' + str(ctime()))        
    
    return model, results

In [19]:
def run_taskload_and_optimisation(df_traffic, time_horizon_start, time_horizon_end, time_interval_length_mins, parameters,
                                  df_tfv=None, df_tfv_configs=None, df_configuration_definition=None):
    
    if df_tfv is None:
        df_tfv = load_tfv_definition() 
        
    if df_tfv_configs is None:
        df_tfv_configs = load_tfv_configs()
        
    if df_configuration_definition is None:
        df_configuration_definition = load_configuration_definition()
        
    tfv_list = list(set([item for sublist in df_configuration_definition.TFVs.to_list() for item in sublist]))
    
    df_sector_taskloads = run_taskload_model_elementry_sectors(df_traffic, time_horizon_start, time_horizon_end,
                                                               time_interval_length_mins=time_interval_length_mins)
    
    df_airspace_taskloads = get_taskloads_by_airspace_df(df_sector_taskloads, tfv_list, df_tfv=df_tfv)

    airspace_taskload_dict = airspace_taskload_df_to_dict(df_airspace_taskloads)

    airspace_to_configs_dict = airspace_to_configs_df_to_dict(df_tfv_configs)
    
    m = build_model(parameters, df_tfv_configs, df_configuration_definition, df_airspace_taskloads, 
                    airspace_taskload_dict, airspace_to_configs_dict)
    
    m, r = solve_model(m, 'cplex', 'neos', timeout_time=120, ratio_gap=0.001, show_working=True)
    
    return m, r

## Solve model

This section will solve the model. First define parameters:

In [43]:
time_interval_length_mins = 15

df_sector_taskloads = run_taskload_model_elementry_sectors(df_traffic, time_horizon_start, time_horizon_end,
                                                           time_interval_length_mins=time_interval_length_mins)

In [73]:
df_sector_taskloads[df_sector_taskloads['elementry_sector'] == 'EFINV']

,elementry_sector,time_interval_start,time_interval_end,Sequencing,Altitude Change,Monitoring,Exit,Entry,total_taskload
13,EFINV,2019-08-29 00:00:00+03:00,2019-08-29 00:05:00+03:00,0.0,0.000000,0.008333,0.000000,0.000,0.008333
27,EFINV,2019-08-29 00:05:00+03:00,2019-08-29 00:10:00+03:00,0.0,0.000000,0.008333,0.000000,0.000,0.008333
41,EFINV,2019-08-29 00:10:00+03:00,2019-08-29 00:15:00+03:00,0.0,0.000000,0.007944,0.008333,0.000,0.016278
55,EFINV,2019-08-29 00:15:00+03:00,2019-08-29 00:20:00+03:00,0.0,0.000000,0.000000,0.000000,0.000,0.000000
69,EFINV,2019-08-29 00:20:00+03:00,2019-08-29 00:25:00+03:00,0.0,0.000000,0.000000,0.000000,0.000,0.000000
83,EFINV,2019-08-29 00:25:00+03:00,2019-08-29 00:30:00+03:00,0.0,0.000000,0.000000,0.000000,0.000,0.000000
97,EFINV,2019-08-29 00:30:00+03:00,2019-08-29 00:35:00+03:00,0.0,0.000000,0.000000,0.000000,0.000,0.000000
111,EFINV,2019-08-29 00:35:00+03:00,2019-08-29 00:40:00+03:00,0.0,0.000000,0.000000,0.000000,0.000,0.000000
125,EFINV,2019-08-29 00:40:00+03:00,2019-08-29 00:45:00+03:00,0.0,0.000000,0.000000,0.000000,0.000,0.000000
139,EFINV,2019-08-29 00:45:00+03:00,2019-08-29 00:50:00+03:00,0.0,0.000000,0.000000,0.000000,0.000,0.000000


In [84]:
def create_rolling_sector_taskload_df(df_sector_taskloads, window=3, center=True, win_type=None):
    
    elementry_sector_list = sorted(list(set(df_sector_taskloads.elementry_sector.to_list())))
    
    out_df = df_sector_taskloads[df_sector_taskloads['elementry_sector'] == 'EFINA'].copy(deep=True)#.reset_index()
    out_df['total_taskload'] = out_df.total_taskload.rolling(window=window, center=center,
                                                             win_type=win_type,min_periods=1).mean()
    
    for sector in elementry_sector_list[1:]:
    
        temp_df =  df_sector_taskloads[df_sector_taskloads['elementry_sector'] == sector].copy(deep=True)
        temp_df['total_taskload'] = temp_df.total_taskload.rolling(window=window, center=center,
                                                                   win_type=win_type,min_periods=1).mean()
        out_df = pd.concat([out_df, temp_df],ignore_index=True)
    
    return out_df#.reset_index()

In [85]:
temp_rolling_df = create_rolling_sector_taskload_df(df_sector_taskloads) #, window=3, center=True, win_type=None)
temp_rolling_df

,elementry_sector,time_interval_start,time_interval_end,Sequencing,Altitude Change,Monitoring,Exit,Entry,total_taskload
0,EFINA,2019-08-29 00:00:00+03:00,2019-08-29 00:05:00+03:00,0.0,0.000000,0.000000,0.000000,0.000,0.000000
1,EFINA,2019-08-29 00:05:00+03:00,2019-08-29 00:10:00+03:00,0.0,0.000000,0.000000,0.000000,0.000,0.000000
2,EFINA,2019-08-29 00:10:00+03:00,2019-08-29 00:15:00+03:00,0.0,0.000000,0.000000,0.000000,0.000,0.000000
3,EFINA,2019-08-29 00:15:00+03:00,2019-08-29 00:20:00+03:00,0.0,0.000000,0.000000,0.000000,0.000,0.000000
4,EFINA,2019-08-29 00:20:00+03:00,2019-08-29 00:25:00+03:00,0.0,0.000000,0.000000,0.000000,0.000,0.000000
5,EFINA,2019-08-29 00:25:00+03:00,2019-08-29 00:30:00+03:00,0.0,0.000000,0.000000,0.000000,0.000,0.000000
6,EFINA,2019-08-29 00:30:00+03:00,2019-08-29 00:35:00+03:00,0.0,0.000000,0.000000,0.000000,0.000,0.019787
7,EFINA,2019-08-29 00:35:00+03:00,2019-08-29 00:40:00+03:00,0.0,0.000000,0.001361,0.000000,0.058,0.045102
8,EFINA,2019-08-29 00:40:00+03:00,2019-08-29 00:45:00+03:00,0.0,0.000000,0.009611,0.008333,0.058,0.049546
9,EFINA,2019-08-29 00:45:00+03:00,2019-08-29 00:50:00+03:00,0.0,0.000000,0.005000,0.008333,0.000,0.029759


In [75]:
temp_rolling_df.total_taskload.sum()

137.97508333333332

In [76]:
temp_rolling_df.rolling_total_taskload.sum()

137.98363888888892

In [44]:
df_airspace_taskloads = get_taskloads_by_airspace_df(df_sector_taskloads, tfv_list, df_tfv=df_tfv)

airspace_taskload_dict = airspace_taskload_df_to_dict(df_airspace_taskloads)

airspace_to_configs_dict = airspace_to_configs_df_to_dict(df_tfv_configs)

In [86]:
df_airspace_taskloads = get_taskloads_by_airspace_df(temp_rolling_df, tfv_list, df_tfv=df_tfv)

airspace_taskload_dict = airspace_taskload_df_to_dict(df_airspace_taskloads)

airspace_to_configs_dict = airspace_to_configs_df_to_dict(df_tfv_configs)

In [82]:
df_airspace_taskloads

,airspace_id,time_interval_start,time_interval_end,Sequencing,Altitude Change,Monitoring,Exit,Entry,total_taskload,rolling_total_taskload
0,EF6GVKN,2019-08-29 00:00:00+03:00,2019-08-29 00:05:00+03:00,0.000000,0.000000,0.008333,0.000000,0.000,0.008333,8.333333e-03
1,EF6GVKN,2019-08-29 00:05:00+03:00,2019-08-29 00:10:00+03:00,0.000000,0.000000,0.008333,0.000000,0.000,0.008333,3.044444e-02
2,EF6GVKN,2019-08-29 00:10:00+03:00,2019-08-29 00:15:00+03:00,0.000000,0.000000,0.008333,0.008333,0.058,0.074667,7.133333e-02
3,EF6GVKN,2019-08-29 00:15:00+03:00,2019-08-29 00:20:00+03:00,0.000000,0.000000,0.015000,0.000000,0.116,0.131000,7.688889e-02
4,EF6GVKN,2019-08-29 00:20:00+03:00,2019-08-29 00:25:00+03:00,0.000000,0.000000,0.025000,0.000000,0.000,0.025000,8.274074e-02
5,EF6GVKN,2019-08-29 00:25:00+03:00,2019-08-29 00:30:00+03:00,0.000000,0.000000,0.017556,0.016667,0.058,0.092222,6.674074e-02
6,EF6GVKN,2019-08-29 00:30:00+03:00,2019-08-29 00:35:00+03:00,0.000000,0.000000,0.016667,0.008333,0.058,0.083000,6.628704e-02
7,EF6GVKN,2019-08-29 00:35:00+03:00,2019-08-29 00:40:00+03:00,0.000000,0.000000,0.015306,0.008333,0.000,0.023639,4.012037e-02
8,EF6GVKN,2019-08-29 00:40:00+03:00,2019-08-29 00:45:00+03:00,0.000000,0.000000,0.005389,0.008333,0.000,0.013722,1.245370e-02
9,EF6GVKN,2019-08-29 00:45:00+03:00,2019-08-29 00:50:00+03:00,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,4.574074e-03


In [83]:
df_airspace_taskloads[df_airspace_taskloads['airspace_id'] == 'EFINALL']

,airspace_id,time_interval_start,time_interval_end,Sequencing,Altitude Change,Monitoring,Exit,Entry,total_taskload,rolling_total_taskload
10080,EFINALL,2019-08-29 00:00:00+03:00,2019-08-29 00:05:00+03:00,0.000000,0.000000,0.008333,0.000000,0.000,0.008333,8.333333e-03
10081,EFINALL,2019-08-29 00:05:00+03:00,2019-08-29 00:10:00+03:00,0.000000,0.000000,0.008333,0.000000,0.000,0.008333,3.044444e-02
10082,EFINALL,2019-08-29 00:10:00+03:00,2019-08-29 00:15:00+03:00,0.000000,0.000000,0.008333,0.008333,0.058,0.074667,7.133333e-02
10083,EFINALL,2019-08-29 00:15:00+03:00,2019-08-29 00:20:00+03:00,0.000000,0.000000,0.015000,0.000000,0.116,0.131000,7.688889e-02
10084,EFINALL,2019-08-29 00:20:00+03:00,2019-08-29 00:25:00+03:00,0.000000,0.000000,0.025000,0.000000,0.000,0.025000,1.045556e-01
10085,EFINALL,2019-08-29 00:25:00+03:00,2019-08-29 00:30:00+03:00,0.000000,0.000000,0.025000,0.016667,0.116,0.157667,9.133333e-02
10086,EFINALL,2019-08-29 00:30:00+03:00,2019-08-29 00:35:00+03:00,0.000000,0.000000,0.025000,0.008333,0.058,0.091333,1.134444e-01
10087,EFINALL,2019-08-29 00:35:00+03:00,2019-08-29 00:40:00+03:00,0.000000,0.000000,0.025000,0.008333,0.058,0.091333,9.466667e-02
10088,EFINALL,2019-08-29 00:40:00+03:00,2019-08-29 00:45:00+03:00,0.000000,0.000000,0.018333,0.025000,0.058,0.101333,8.855556e-02
10089,EFINALL,2019-08-29 00:45:00+03:00,2019-08-29 00:50:00+03:00,0.000000,0.000000,0.006667,0.008333,0.058,0.073000,6.088889e-02


In [87]:
parameters = {
    'double_controller_taskload_limit': 0.6,
    'single_controller_taskload_limit': 0.3, 
    'time_interval_length_mins': time_interval_length_mins,
    'number_time_intervals': int(24/(time_interval_length_mins/60)),
    'min_reconfig_period': 3,
    'epsilon': 0.00001,
    'weight_1': 0.5,
    'weight_2': 0.3
}
parameters

{'double_controller_taskload_limit': 0.6,
 'single_controller_taskload_limit': 0.3,
 'time_interval_length_mins': 5,
 'number_time_intervals': 288,
 'min_reconfig_period': 3,
 'epsilon': 1e-05,
 'weight_1': 0.5,
 'weight_2': 0.3}

In [20]:
#time_interval_length_mins = 15#

#m, r = run_taskload_and_optimisation(df_traffic, time_horizon_start, time_horizon_end, time_interval_length_mins, 
#                                     parameters)

Run start time: Fri Sep 20 15:15:57 2019
Run finish time: Fri Sep 20 15:16:27 2019


In [88]:
m = build_model(parameters, df_tfv_configs, df_configuration_definition, df_airspace_taskloads, 
                airspace_taskload_dict, airspace_to_configs_dict)

In [89]:
m, r = solve_model(m, 'cplex', 'neos', timeout_time=120, ratio_gap=0.001, show_working=True)

Run start time: Mon Sep 23 13:59:32 2019
Run finish time: Mon Sep 23 14:00:01 2019


In [58]:
#path = r'C:\repos\mat.deloitte.ansfinland.anpo_sectorisation_poc\solvers\cbc\bin\cbc.exe'
#m, r = solve_model(m, 'cbc', path, timeout_time=60, ratio_gap=0.001, show_working=True)

In [22]:
r

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 37796, 'Number of variables': 18953, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'CPLEX 12.9.0.0\\x3a optimal integer solution; objective 175.90000000001015; 7297 MIP simplex iterations; 0 branch-and-bound nodes', 'Termination condition': 'optimal', 'Id': 2}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [ ]:
# no r: 129 sector hours, 32 diff configs, n = 80
# with r: 129 sector hours, 23 diff configs, n = 80

In [42]:
df_sectorised_taskload = model_to_sector_configuration_df(m, time_interval_length_mins)

df_graph_data = create_taskload_by_tfv_data(df_sector_taskloads, df_sectorised_taskload, time_interval_length_mins, 
                                        df_tfv = df_tfv, sort_taskloads = True)

print(report_results(df_sectorised_taskload, num_airspaces_in_config_dict))

create_taskload_by_tfv_bar_chart(df_sectorised_taskload, df_graph_data)

{'sector_open_hours': 32.25, 'configuration_changes': 23, 'single_controller_controlling_hours': 12.25, 'double_controller_controlling_hours': 20.0}


In [48]:
df_sectorised_taskload = model_to_sector_configuration_df(m, time_interval_length_mins)

df_graph_data = create_taskload_by_tfv_data(df_sector_taskloads, df_sectorised_taskload, time_interval_length_mins, 
                                        df_tfv = df_tfv, sort_taskloads = True)

print(report_results(df_sectorised_taskload, num_airspaces_in_config_dict))

create_taskload_by_tfv_bar_chart(df_sectorised_taskload, df_graph_data)

{'sector_open_hours': 37.583333333333336, 'configuration_changes': 46, 'single_controller_controlling_hours': 19.666666666666664, 'double_controller_controlling_hours': 17.916666666666668}


In [90]:
df_sectorised_taskload = model_to_sector_configuration_df(m, time_interval_length_mins)

df_graph_data = create_taskload_by_tfv_data(temp_rolling_df, df_sectorised_taskload, time_interval_length_mins, 
                                        df_tfv = df_tfv, sort_taskloads = True)

print(report_results(df_sectorised_taskload, num_airspaces_in_config_dict))

create_taskload_by_tfv_bar_chart(df_sectorised_taskload, df_graph_data)

{'sector_open_hours': 32.16666666666667, 'configuration_changes': 25, 'single_controller_controlling_hours': 12.083333333333334, 'double_controller_controlling_hours': 20.08333333333334}


In [96]:
df_sector_taskloads[df_sector_taskloads['time_interval_start'] == mins_after_midnight_to_timestamp(5*155)]

,elementry_sector,time_interval_start,time_interval_end,Sequencing,Altitude Change,Monitoring,Exit,Entry,total_taskload
2170,EFINA,2019-08-29 12:55:00+03:00,2019-08-29 13:00:00+03:00,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2171,EFINB,2019-08-29 12:55:00+03:00,2019-08-29 13:00:00+03:00,0.00000,0.02333,0.01183,0.00833,0.00000,0.04350
2172,EFINC,2019-08-29 12:55:00+03:00,2019-08-29 13:00:00+03:00,0.00000,0.02333,0.02150,0.00000,0.05800,0.10283
2173,EFIND,2019-08-29 12:55:00+03:00,2019-08-29 13:00:00+03:00,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2174,EFINE,2019-08-29 12:55:00+03:00,2019-08-29 13:00:00+03:00,0.00000,0.02333,0.01386,0.00833,0.05800,0.10353
2175,EFINF,2019-08-29 12:55:00+03:00,2019-08-29 13:00:00+03:00,0.00000,0.00000,0.04475,0.00000,0.17400,0.21875
2176,EFING,2019-08-29 12:55:00+03:00,2019-08-29 13:00:00+03:00,0.00000,0.00000,0.00833,0.00000,0.00000,0.00833
2177,EFINH,2019-08-29 12:55:00+03:00,2019-08-29 13:00:00+03:00,0.00000,0.00000,0.00050,0.00000,0.05800,0.05850
2178,EFINJ,2019-08-29 12:55:00+03:00,2019-08-29 13:00:00+03:00,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2179,EFINK,2019-08-29 12:55:00+03:00,2019-08-29 13:00:00+03:00,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


In [97]:
temp_rolling_df[temp_rolling_df['time_interval_start'] == mins_after_midnight_to_timestamp(5*155)]

,elementry_sector,time_interval_start,time_interval_end,Sequencing,Altitude Change,Monitoring,Exit,Entry,total_taskload
155,EFINA,2019-08-29 12:55:00+03:00,2019-08-29 13:00:00+03:00,0.00000,0.00000,0.00000,0.00000,0.00000,0.01126
443,EFINB,2019-08-29 12:55:00+03:00,2019-08-29 13:00:00+03:00,0.00000,0.02333,0.01183,0.00833,0.00000,0.08001
731,EFINC,2019-08-29 12:55:00+03:00,2019-08-29 13:00:00+03:00,0.00000,0.02333,0.02150,0.00000,0.05800,0.06867
1019,EFIND,2019-08-29 12:55:00+03:00,2019-08-29 13:00:00+03:00,0.00000,0.00000,0.00000,0.00000,0.00000,0.01972
1307,EFINE,2019-08-29 12:55:00+03:00,2019-08-29 13:00:00+03:00,0.00000,0.02333,0.01386,0.00833,0.05800,0.16832
1595,EFINF,2019-08-29 12:55:00+03:00,2019-08-29 13:00:00+03:00,0.00000,0.00000,0.04475,0.00000,0.17400,0.21432
1883,EFING,2019-08-29 12:55:00+03:00,2019-08-29 13:00:00+03:00,0.00000,0.00000,0.00833,0.00000,0.00000,0.07392
2171,EFINH,2019-08-29 12:55:00+03:00,2019-08-29 13:00:00+03:00,0.00000,0.00000,0.00050,0.00000,0.05800,0.03132
2459,EFINJ,2019-08-29 12:55:00+03:00,2019-08-29 13:00:00+03:00,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2747,EFINK,2019-08-29 12:55:00+03:00,2019-08-29 13:00:00+03:00,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


In [98]:
df_sectorised_taskload

,Configuration,Start,End,Single_Controller_Controlling_Hours,Double_Controller_Controlling_Hours
0,CONF1,2019-08-29 00:00:00+03:00,2019-08-29 03:20:00+03:00,2.25000,1.08333
1,CONF2I,2019-08-29 03:20:00+03:00,2019-08-29 03:35:00+03:00,0.33333,0.16667
2,CONF1,2019-08-29 03:35:00+03:00,2019-08-29 05:00:00+03:00,0.25000,1.16667
3,CONF2A,2019-08-29 05:00:00+03:00,2019-08-29 05:15:00+03:00,0.33333,0.16667
4,CONF1,2019-08-29 05:15:00+03:00,2019-08-29 06:25:00+03:00,0.00000,1.16667
5,CONF2D,2019-08-29 06:25:00+03:00,2019-08-29 07:00:00+03:00,0.58333,0.58333
6,CONF1,2019-08-29 07:00:00+03:00,2019-08-29 10:05:00+03:00,0.00000,3.08333
7,CONF2E,2019-08-29 10:05:00+03:00,2019-08-29 10:35:00+03:00,0.16667,0.83333
8,CONF2B,2019-08-29 10:35:00+03:00,2019-08-29 11:00:00+03:00,0.00000,0.83333
9,CONF3A,2019-08-29 11:00:00+03:00,2019-08-29 11:35:00+03:00,0.58333,1.16667


In [164]:
counter = 0
for i, j in m.q.get_values().items():
    print(i)
    print(j)
    print(' ')
    counter += j
counter

0
1.0
 
1
1.0
 
2
1.0
 
3
1.0
 
4
1.0
 
5
1.0
 
6
1.0
 
7
1.0
 
8
1.0
 
9
1.0
 
10
1.0
 
11
1.0
 
12
1.0
 
13
2.0
 
14
2.0
 
15
1.0
 
16
1.0
 
17
1.0
 
18
1.0
 
19
2.0
 
20
2.0
 
21
1.0
 
22
1.0
 
23
1.0
 
24
1.0
 
25
1.0
 
26
2.0
 
27
2.0
 
28
1.0
 
29
1.0
 
30
1.0
 
31
1.0
 
32
1.0
 
33
1.0
 
34
1.0
 
35
1.0
 
36
1.0
 
37
1.0
 
38
1.0
 
39
1.0
 
40
2.0
 
41
2.0
 
42
2.0
 
43
2.0
 
44
3.0
 
45
3.0
 
46
2.0
 
47
2.0
 
48
2.0
 
49
2.0
 
50
2.0
 
51
2.0
 
52
2.0
 
53
2.0
 
54
2.0
 
55
2.0
 
56
2.0
 
57
2.0
 
58
2.0
 
59
2.0
 
60
2.0
 
61
1.0
 
62
1.0
 
63
1.0
 
64
2.0
 
65
2.0
 
66
2.0
 
67
2.0
 
68
1.0
 
69
1.0
 
70
1.0
 
71
1.0
 
72
1.0
 
73
1.0
 
74
1.0
 
75
1.0
 
76
2.0
 
77
2.0
 
78
1.0
 
79
1.0
 
80
1.0
 
81
1.0
 
82
1.0
 
83
1.0
 
84
2.0
 
85
2.0
 
86
1.0
 
87
1.0
 
88
1.0
 
89
1.0
 
90
1.0
 
91
1.0
 
92
1.0
 
93
1.0
 
94
1.0
 
95
1.0
 
96
1.0
 
97
1.0
 
98
1.0
 
99
1.0
 
100
1.0
 
101
1.0
 
102
1.0
 
103
1.0
 
104
1.0
 
105
1.0
 
106
1.0
 
107
1.0
 
108
1.0
 
109
1.0
 
110
1.0
 


325.0

In [57]:
counter = 0
for i, j in m.n.get_values().items():
    print(i)
    print(j)
    print(' ')
    counter += j
counter

('EF6AF', 0)
0.0
 
('EF6AF', 1)
0.0
 
('EF6AF', 2)
0.0
 
('EF6AF', 3)
0.0
 
('EF6AF', 4)
0.0
 
('EF6AF', 5)
0.0
 
('EF6AF', 6)
0.0
 
('EF6AF', 7)
0.0
 
('EF6AF', 8)
0.0
 
('EF6AF', 9)
0.0
 
('EF6AF', 10)
0.0
 
('EF6AF', 11)
0.0
 
('EF6AF', 12)
0.0
 
('EF6AF', 13)
0.0
 
('EF6AF', 14)
0.0
 
('EF6AF', 15)
0.0
 
('EF6AF', 16)
0.0
 
('EF6AF', 17)
0.0
 
('EF6AF', 18)
0.0
 
('EF6AF', 19)
0.0
 
('EF6AF', 20)
0.0
 
('EF6AF', 21)
0.0
 
('EF6AF', 22)
0.0
 
('EF6AF', 23)
0.0
 
('EF6AF', 24)
0.0
 
('EF6AF', 25)
0.0
 
('EF6AF', 26)
0.0
 
('EF6AF', 27)
0.0
 
('EF6AF', 28)
0.0
 
('EF6AF', 29)
0.0
 
('EF6AF', 30)
0.0
 
('EF6AF', 31)
0.0
 
('EF6AF', 32)
0.0
 
('EF6AF', 33)
0.0
 
('EF6AF', 34)
0.0
 
('EF6AF', 35)
0.0
 
('EF6AF', 36)
0.0
 
('EF6AF', 37)
0.0
 
('EF6AF', 38)
0.0
 
('EF6AF', 39)
0.0
 
('EF6AF', 40)
0.0
 
('EF6AF', 41)
0.0
 
('EF6AF', 42)
0.0
 
('EF6AF', 43)
0.0
 
('EF6AF', 44)
0.0
 
('EF6AF', 45)
0.0
 
('EF6AF', 46)
0.0
 
('EF6AF', 47)
0.0
 
('EF6AF', 48)
0.0
 
('EF6AF', 49)
0.0
 
('EF6AF', 

 
('EF6EJ', 92)
0.0
 
('EF6EJ', 93)
0.0
 
('EF6EJ', 94)
0.0
 
('EF6EJ', 95)
0.0
 
('EF12XHJ', 0)
0.0
 
('EF12XHJ', 1)
0.0
 
('EF12XHJ', 2)
0.0
 
('EF12XHJ', 3)
0.0
 
('EF12XHJ', 4)
0.0
 
('EF12XHJ', 5)
0.0
 
('EF12XHJ', 6)
0.0
 
('EF12XHJ', 7)
0.0
 
('EF12XHJ', 8)
0.0
 
('EF12XHJ', 9)
0.0
 
('EF12XHJ', 10)
0.0
 
('EF12XHJ', 11)
0.0
 
('EF12XHJ', 12)
0.0
 
('EF12XHJ', 13)
0.0
 
('EF12XHJ', 14)
0.0
 
('EF12XHJ', 15)
0.0
 
('EF12XHJ', 16)
0.0
 
('EF12XHJ', 17)
0.0
 
('EF12XHJ', 18)
0.0
 
('EF12XHJ', 19)
0.0
 
('EF12XHJ', 20)
0.0
 
('EF12XHJ', 21)
0.0
 
('EF12XHJ', 22)
0.0
 
('EF12XHJ', 23)
0.0
 
('EF12XHJ', 24)
0.0
 
('EF12XHJ', 25)
0.0
 
('EF12XHJ', 26)
0.0
 
('EF12XHJ', 27)
0.0
 
('EF12XHJ', 28)
0.0
 
('EF12XHJ', 29)
0.0
 
('EF12XHJ', 30)
0.0
 
('EF12XHJ', 31)
0.0
 
('EF12XHJ', 32)
0.0
 
('EF12XHJ', 33)
0.0
 
('EF12XHJ', 34)
0.0
 
('EF12XHJ', 35)
0.0
 
('EF12XHJ', 36)
0.0
 
('EF12XHJ', 37)
0.0
 
('EF12XHJ', 38)
0.0
 
('EF12XHJ', 39)
0.0
 
('EF12XHJ', 40)
0.0
 
('EF12XHJ', 41)
0.0
 
('EF

0.0
 
('EF4GVHJ', 86)
0.0
 
('EF4GVHJ', 87)
0.0
 
('EF4GVHJ', 88)
0.0
 
('EF4GVHJ', 89)
0.0
 
('EF4GVHJ', 90)
0.0
 
('EF4GVHJ', 91)
0.0
 
('EF4GVHJ', 92)
0.0
 
('EF4GVHJ', 93)
0.0
 
('EF4GVHJ', 94)
0.0
 
('EF4GVHJ', 95)
0.0
 
('EF4HJNV', 0)
0.0
 
('EF4HJNV', 1)
0.0
 
('EF4HJNV', 2)
0.0
 
('EF4HJNV', 3)
0.0
 
('EF4HJNV', 4)
0.0
 
('EF4HJNV', 5)
0.0
 
('EF4HJNV', 6)
0.0
 
('EF4HJNV', 7)
0.0
 
('EF4HJNV', 8)
0.0
 
('EF4HJNV', 9)
0.0
 
('EF4HJNV', 10)
0.0
 
('EF4HJNV', 11)
0.0
 
('EF4HJNV', 12)
0.0
 
('EF4HJNV', 13)
0.0
 
('EF4HJNV', 14)
0.0
 
('EF4HJNV', 15)
0.0
 
('EF4HJNV', 16)
0.0
 
('EF4HJNV', 17)
0.0
 
('EF4HJNV', 18)
0.0
 
('EF4HJNV', 19)
0.0
 
('EF4HJNV', 20)
0.0
 
('EF4HJNV', 21)
0.0
 
('EF4HJNV', 22)
0.0
 
('EF4HJNV', 23)
0.0
 
('EF4HJNV', 24)
0.0
 
('EF4HJNV', 25)
0.0
 
('EF4HJNV', 26)
0.0
 
('EF4HJNV', 27)
0.0
 
('EF4HJNV', 28)
0.0
 
('EF4HJNV', 29)
0.0
 
('EF4HJNV', 30)
0.0
 
('EF4HJNV', 31)
0.0
 
('EF4HJNV', 32)
0.0
 
('EF4HJNV', 33)
0.0
 
('EF4HJNV', 34)
0.0
 
('EF4HJNV', 35

 
('EF2FG', 60)
0.0
 
('EF2FG', 61)
0.0
 
('EF2FG', 62)
0.0
 
('EF2FG', 63)
0.0
 
('EF2FG', 64)
0.0
 
('EF2FG', 65)
0.0
 
('EF2FG', 66)
0.0
 
('EF2FG', 67)
0.0
 
('EF2FG', 68)
0.0
 
('EF2FG', 69)
0.0
 
('EF2FG', 70)
0.0
 
('EF2FG', 71)
0.0
 
('EF2FG', 72)
0.0
 
('EF2FG', 73)
0.0
 
('EF2FG', 74)
0.0
 
('EF2FG', 75)
0.0
 
('EF2FG', 76)
0.0
 
('EF2FG', 77)
0.0
 
('EF2FG', 78)
0.0
 
('EF2FG', 79)
0.0
 
('EF2FG', 80)
0.0
 
('EF2FG', 81)
0.0
 
('EF2FG', 82)
0.0
 
('EF2FG', 83)
0.0
 
('EF2FG', 84)
0.0
 
('EF2FG', 85)
0.0
 
('EF2FG', 86)
0.0
 
('EF2FG', 87)
0.0
 
('EF2FG', 88)
0.0
 
('EF2FG', 89)
0.0
 
('EF2FG', 90)
0.0
 
('EF2FG', 91)
0.0
 
('EF2FG', 92)
0.0
 
('EF2FG', 93)
0.0
 
('EF2FG', 94)
0.0
 
('EF2FG', 95)
0.0
 
('EF4KLMN', 0)
0.0
 
('EF4KLMN', 1)
0.0
 
('EF4KLMN', 2)
0.0
 
('EF4KLMN', 3)
0.0
 
('EF4KLMN', 4)
0.0
 
('EF4KLMN', 5)
0.0
 
('EF4KLMN', 6)
0.0
 
('EF4KLMN', 7)
0.0
 
('EF4KLMN', 8)
0.0
 
('EF4KLMN', 9)
0.0
 
('EF4KLMN', 10)
0.0
 
('EF4KLMN', 11)
0.0
 
('EF4KLMN', 12)
0.0
 
('

 
('EF8ADKN', 8)
0.0
 
('EF8ADKN', 9)
0.0
 
('EF8ADKN', 10)
0.0
 
('EF8ADKN', 11)
0.0
 
('EF8ADKN', 12)
0.0
 
('EF8ADKN', 13)
0.0
 
('EF8ADKN', 14)
0.0
 
('EF8ADKN', 15)
0.0
 
('EF8ADKN', 16)
0.0
 
('EF8ADKN', 17)
0.0
 
('EF8ADKN', 18)
0.0
 
('EF8ADKN', 19)
0.0
 
('EF8ADKN', 20)
0.0
 
('EF8ADKN', 21)
0.0
 
('EF8ADKN', 22)
0.0
 
('EF8ADKN', 23)
0.0
 
('EF8ADKN', 24)
0.0
 
('EF8ADKN', 25)
0.0
 
('EF8ADKN', 26)
0.0
 
('EF8ADKN', 27)
0.0
 
('EF8ADKN', 28)
0.0
 
('EF8ADKN', 29)
0.0
 
('EF8ADKN', 30)
0.0
 
('EF8ADKN', 31)
0.0
 
('EF8ADKN', 32)
0.0
 
('EF8ADKN', 33)
0.0
 
('EF8ADKN', 34)
0.0
 
('EF8ADKN', 35)
0.0
 
('EF8ADKN', 36)
0.0
 
('EF8ADKN', 37)
0.0
 
('EF8ADKN', 38)
0.0
 
('EF8ADKN', 39)
0.0
 
('EF8ADKN', 40)
0.0
 
('EF8ADKN', 41)
0.0
 
('EF8ADKN', 42)
0.0
 
('EF8ADKN', 43)
0.0
 
('EF8ADKN', 44)
0.0
 
('EF8ADKN', 45)
0.0
 
('EF8ADKN', 46)
0.0
 
('EF8ADKN', 47)
0.0
 
('EF8ADKN', 48)
0.0
 
('EF8ADKN', 49)
0.0
 
('EF8ADKN', 50)
0.0
 
('EF8ADKN', 51)
0.0
 
('EF8ADKN', 52)
0.0
 
('EF8ADKN'

 
('EF8GN', 45)
0.0
 
('EF8GN', 46)
0.0
 
('EF8GN', 47)
0.0
 
('EF8GN', 48)
0.0
 
('EF8GN', 49)
0.0
 
('EF8GN', 50)
0.0
 
('EF8GN', 51)
0.0
 
('EF8GN', 52)
1.0
 
('EF8GN', 53)
1.0
 
('EF8GN', 54)
1.0
 
('EF8GN', 55)
0.0
 
('EF8GN', 56)
0.0
 
('EF8GN', 57)
0.0
 
('EF8GN', 58)
1.0
 
('EF8GN', 59)
1.0
 
('EF8GN', 60)
1.0
 
('EF8GN', 61)
0.0
 
('EF8GN', 62)
0.0
 
('EF8GN', 63)
0.0
 
('EF8GN', 64)
0.0
 
('EF8GN', 65)
0.0
 
('EF8GN', 66)
0.0
 
('EF8GN', 67)
0.0
 
('EF8GN', 68)
0.0
 
('EF8GN', 69)
0.0
 
('EF8GN', 70)
0.0
 
('EF8GN', 71)
0.0
 
('EF8GN', 72)
0.0
 
('EF8GN', 73)
0.0
 
('EF8GN', 74)
0.0
 
('EF8GN', 75)
0.0
 
('EF8GN', 76)
0.0
 
('EF8GN', 77)
0.0
 
('EF8GN', 78)
0.0
 
('EF8GN', 79)
0.0
 
('EF8GN', 80)
0.0
 
('EF8GN', 81)
0.0
 
('EF8GN', 82)
0.0
 
('EF8GN', 83)
0.0
 
('EF8GN', 84)
0.0
 
('EF8GN', 85)
0.0
 
('EF8GN', 86)
0.0
 
('EF8GN', 87)
0.0
 
('EF8GN', 88)
0.0
 
('EF8GN', 89)
0.0
 
('EF8GN', 90)
0.0
 
('EF8GN', 91)
0.0
 
('EF8GN', 92)
0.0
 
('EF8GN', 93)
0.0
 
('EF8GN', 94)
0.0


76.0

In [50]:
sum([v for k, v in m.n.get_values().items() 
     if 10 <= k[1] <= 20 and k[0] in get_config_tfvs_list(df_configuration_definition, 'CONF2E')])

1.0

In [71]:
#m.r.pprint()

In [201]:
model_to_sector_configuration_df(m, time_interval_length_mins)

,Configuration,Start,End
0,CONF1,2019-08-29 00:00:00+03:00,2019-08-29 03:10:00+03:00
1,CONF2E,2019-08-29 03:10:00+03:00,2019-08-29 03:30:00+03:00
2,CONF2D,2019-08-29 03:30:00+03:00,2019-08-29 03:35:00+03:00
3,CONF1,2019-08-29 03:35:00+03:00,2019-08-29 04:25:00+03:00
4,CONF2,2019-08-29 04:25:00+03:00,2019-08-29 04:30:00+03:00
5,CONF1,2019-08-29 04:30:00+03:00,2019-08-29 05:05:00+03:00
6,CONF2E,2019-08-29 05:05:00+03:00,2019-08-29 05:15:00+03:00
7,CONF1,2019-08-29 05:15:00+03:00,2019-08-29 05:50:00+03:00
8,CONF2I,2019-08-29 05:50:00+03:00,2019-08-29 05:55:00+03:00
9,CONF1,2019-08-29 05:55:00+03:00,2019-08-29 06:25:00+03:00


In [131]:
counter = 0
for i, j in m.s.get_values().items():
    print(i)
    print(j)
    print(' ')
    counter += j
counter

0
0.0
 
1
0.0
 
2
0.0
 
3
0.0
 
4
0.0
 
5
0.0
 
6
0.0
 
7
0.0
 
8
0.0
 
9
0.0
 
10
0.0
 
11
0.0
 
12
0.0
 
13
0.0
 
14
0.0
 
15
0.0
 
16
0.0
 
17
0.0
 
18
0.0
 
19
0.0
 
20
0.0
 
21
0.0
 
22
0.0
 
23
0.0
 
24
0.0
 
25
0.0
 
26
0.0
 
27
0.0
 
28
0.0
 
29
0.0
 
30
0.0
 
31
0.0
 
32
0.0
 
33
0.0
 
34
0.0
 
35
0.0
 
36
0.0
 
37
1.0
 
38
0.0
 
39
0.0
 
40
0.0
 
41
0.0
 
42
1.0
 
43
0.0
 
44
0.0
 
45
0.0
 
46
0.0
 
47
0.0
 
48
0.0
 
49
1.0
 
50
0.0
 
51
0.0
 
52
0.0
 
53
1.0
 
54
0.0
 
55
0.0
 
56
0.0
 
57
0.0
 
58
1.0
 
59
0.0
 
60
0.0
 
61
0.0
 
62
1.0
 
63
0.0
 
64
0.0
 
65
0.0
 
66
0.0
 
67
1.0
 
68
0.0
 
69
0.0
 
70
0.0
 
71
1.0
 
72
0.0
 
73
0.0
 
74
0.0
 
75
0.0
 
76
1.0
 
77
0.0
 
78
0.0
 
79
0.0
 
80
0.0
 
81
0.0
 
82
1.0
 
83
0.0
 
84
0.0
 
85
0.0
 
86
1.0
 
87
0.0
 
88
0.0
 
89
0.0
 
90
0.0
 
91
0.0
 
92
0.0
 
93
1.0
 
94
0.0
 
95
0.0
 
96
0.0
 
97
1.0
 
98
0.0
 
99
0.0
 
100
0.0
 
101
1.0
 
102
0.0
 
103
0.0
 
104
0.0
 
105
1.0
 
106
0.0
 
107
0.0
 
108
0.0
 
109
0.0
 
110
1.0
 


41.0

In [132]:
counter = 0
for i, j in m.r.get_values().items():
    if j == 1:
        counter += j
        print(i)
        print(j)
        print(' ')
    
counter

('CONF1', 37)
1.0
 
('CONF1', 42)
1.0
 
('CONF1', 49)
1.0
 
('CONF1', 53)
1.0
 
('CONF1', 58)
1.0
 
('CONF1', 62)
1.0
 
('CONF1', 67)
1.0
 
('CONF1', 71)
1.0
 
('CONF1', 76)
1.0
 
('CONF1', 86)
1.0
 
('CONF1', 93)
1.0
 
('CONF1', 97)
1.0
 
('CONF1', 101)
1.0
 
('CONF1', 105)
1.0
 
('CONF1', 110)
1.0
 
('CONF1', 116)
1.0
 
('CONF1', 121)
1.0
 
('CONF1', 182)
1.0
 
('CONF1', 192)
1.0
 
('CONF1', 205)
1.0
 
('CONF1', 209)
1.0
 
('CONF1', 213)
1.0
 
('CONF1', 223)
1.0
 
('CONF1', 234)
1.0
 
('CONF1', 250)
1.0
 
('CONF1', 254)
1.0
 
('CONF2', 101)
1.0
 
('CONF2', 105)
1.0
 
('CONF2B', 37)
1.0
 
('CONF2B', 42)
1.0
 
('CONF2E', 110)
1.0
 
('CONF2E', 116)
1.0
 
('CONF2E', 196)
1.0
 
('CONF2E', 201)
1.0
 
('CONF2E', 209)
1.0
 
('CONF2E', 213)
1.0
 
('CONF2F', 49)
1.0
 
('CONF2F', 53)
1.0
 
('CONF2F', 174)
1.0
 
('CONF2F', 178)
1.0
 
('CONF2F', 223)
1.0
 
('CONF2F', 228)
1.0
 
('CONF2H', 58)
1.0
 
('CONF2H', 62)
1.0
 
('CONF2H', 67)
1.0
 
('CONF2H', 71)
1.0
 
('CONF2H', 93)
1.0
 
('CONF2H', 97)


82.0

In [52]:
def num_double_controller_sectors(model, df_configuration_definition, time_interval_start, time_interval_end, config):
    
    return sum([v for k, v in m.n.get_values().items() 
                if time_interval_start <= k[1] <= time_interval_end 
                and k[0] in get_config_tfvs_list(df_configuration_definition, config)])

In [53]:
num_double_controller_sectors(m, df_configuration_definition, 0, 95, 'CONF1')

32.0

In [39]:
def model_to_sector_configuration_df(model, time_interval_length_mins):
    
    # get the configs in a list sorted by time
    data_list = []
    for i, j in model.p.get_values().items():
        if j == 1:
            data_list.append((i[1], i[0]))
    data_list = sorted(data_list, key=lambda x: x[0])
    
    # iterate through the list and put data into accepted format
    data_list2 = []
    time_start = pd.Timestamp('29/08/2019', tz='Europe/Helsinki')
    last_config = data_list[0][1]
    last_time_interval = 0
    for tup in data_list:
        config = tup[1]
        if config != last_config:
            tis = last_time_interval
            tis_ts = time_start + pd.Timedelta(minutes=time_interval_length_mins * tis)
            tie = tup[0]
            tie_ts = time_start + pd.Timedelta(minutes=time_interval_length_mins * tie)
            data_list2.append([last_config, tis_ts, tie_ts])
            last_time_interval = tie
            last_config = config
    tis = last_time_interval
    tis_ts = time_start + pd.Timedelta(minutes=time_interval_length_mins * tis)
    tie = tup[0]
    tie_ts = time_start + pd.Timedelta(minutes=time_interval_length_mins * (tie+1))
    data_list2.append([last_config, tis_ts, tie_ts])
    
    df_sector_configuration = pd.DataFrame(data_list2, columns=['Configuration', 'Start', 'End'])
    
    # get number of double controller sectors in each config period
    temp_list = [timestamp_to_mins_after_midnight(df_sector_configuration.at[idx,'Start'])/time_interval_length_mins 
                 for idx in df_sector_configuration.index] + [24*(60/time_interval_length_mins)]
    temp_list_2 = [(temp_list[i],temp_list[i+1]-1) for i in range(len(temp_list)-1)]
    temp_dict_m = {i: 0  for i in temp_list_2}
    temp_dict_n = {i: 0  for i in temp_list_2}
    for i, j in model.m.get_values().items():
        if j == 1:
            for k in temp_dict_m.keys():
                if k[0] <= i[1] <= k[1]:
                    if model.n[i] == 1:
                        temp_dict_n[k] += 1
                    else:
                        temp_dict_m[k] += 1
                    break
    temp_list_3 = []
    for k in sorted(temp_dict_m.keys(), key=lambda x: x[0]):
        temp_list_3.append(temp_dict_m[k])
    temp_list_4 = [i*(time_interval_length_mins/60) for i in temp_list_3]
    df_sector_configuration['Single_Controller_Controlling_Hours'] = temp_list_4
    
    temp_list_5 = []
    for k in sorted(temp_dict_n.keys(), key=lambda x: x[0]):
        temp_list_5.append(temp_dict_n[k])
    temp_list_6 = [i*(time_interval_length_mins/60) for i in temp_list_5]
    df_sector_configuration['Double_Controller_Controlling_Hours'] = temp_list_6
    
    
    return df_sector_configuration

In [40]:
d = model_to_sector_configuration_df(m, 15)
d

,Configuration,Start,End,Single_Controller_Controlling_Hours,Double_Controller_Controlling_Hours
0,CONF1,2019-08-29 00:00:00+03:00,2019-08-29 03:15:00+03:00,2.25,1.00
1,CONF2F,2019-08-29 03:15:00+03:00,2019-08-29 03:30:00+03:00,0.25,0.25
2,CONF1,2019-08-29 03:30:00+03:00,2019-08-29 05:00:00+03:00,0.25,1.25
3,CONF2F,2019-08-29 05:00:00+03:00,2019-08-29 05:15:00+03:00,0.25,0.25
4,CONF1,2019-08-29 05:15:00+03:00,2019-08-29 06:30:00+03:00,0.00,1.25
5,CONF2G,2019-08-29 06:30:00+03:00,2019-08-29 07:00:00+03:00,0.50,0.50
6,CONF1,2019-08-29 07:00:00+03:00,2019-08-29 10:00:00+03:00,0.00,3.00
7,CONF2D,2019-08-29 10:00:00+03:00,2019-08-29 10:30:00+03:00,0.50,0.50
8,CONF2J,2019-08-29 10:30:00+03:00,2019-08-29 11:00:00+03:00,0.00,1.00
9,CONF3A,2019-08-29 11:00:00+03:00,2019-08-29 11:30:00+03:00,0.50,1.00


[(0.0, 12.0),
 (13.0, 14.0),
 (15.0, 18.0),
 (19.0, 20.0),
 (21.0, 25.0),
 (26.0, 27.0),
 (28.0, 39.0),
 (40.0, 41.0),
 (42.0, 43.0),
 (44.0, 45.0),
 (46.0, 50.0),
 (51.0, 55.0),
 (56.0, 60.0),
 (61.0, 63.0),
 (64.0, 67.0),
 (68.0, 75.0),
 (76.0, 77.0),
 (78.0, 83.0),
 (84.0, 85.0),
 (86.0, 95.0)]

In [161]:
temp_list = [timestamp_to_mins_after_midnight(d.at[idx,'Start'])/time_interval_length_mins 
             for idx in d.index] + [24*(60/time_interval_length_mins)]
temp_list_2 = [(temp_list[i],temp_list[i+1]-1) for i in range(len(temp_list)-1)]
temp_list_2
temp_dict = {i: 0  for i in temp_list_2}
for i, j in m.m.get_values().items():
    if j == 1:
        for k in temp_dict.keys():
            if k[0] <= i[1] <= k[1]:
                temp_dict[k] += 1
                break
temp_list_3 = []
for k in sorted(temp_dict.keys(), key=lambda x: x[0]):
    temp_list_3.append(temp_dict[k])

In [163]:
sum(temp_list_3)

133

In [152]:
[[i[0]*(time_interval_length_mins/60)] for i in temp_dict_3]

[[1.0],
 [0.25],
 [0.75],
 [0.25],
 [1.25],
 [0.5],
 [3.0],
 [0.5],
 [1.0],
 [1.0],
 [1.75],
 [2.25],
 [1.75],
 [0.75],
 [1.0],
 [2.0],
 [0.5],
 [0.25],
 [0.25],
 [0.0]]

In [41]:
def report_results(df_sector_configuration, num_airspaces_in_config_dict):
    
    sector_open_hours = 0
    single_controller_controlling_hours = 0
    double_controller_controlling_hours = 0
    configuration_changes = max(df_sector_configuration.index)
    
    for idx in df_sector_configuration.index:
        start = df_sector_configuration.at[idx, 'Start']
        end = df_sector_configuration.at[idx, 'End']
        time_delta_hours = pd.Timedelta(end-start).total_seconds()/3600
        config = df_sector_configuration.at[idx, 'Configuration']
        
        sector_open_hours += time_delta_hours * num_airspaces_in_config_dict[config]
        single_controller_controlling_hours += df_sector_configuration.at[idx, 'Single_Controller_Controlling_Hours']
        double_controller_controlling_hours += df_sector_configuration.at[idx, 'Double_Controller_Controlling_Hours']
        
    out_dict = {'sector_open_hours': sector_open_hours,
                'configuration_changes': configuration_changes,
                'single_controller_controlling_hours': single_controller_controlling_hours,
                'double_controller_controlling_hours': double_controller_controlling_hours
               }
    
    return out_dict

In [170]:
d

,Configuration,Start,End,double_controller_hours
0,CONF1,2019-08-29 00:00:00+03:00,2019-08-29 03:15:00+03:00,1.00
1,CONF2D,2019-08-29 03:15:00+03:00,2019-08-29 03:45:00+03:00,0.25
2,CONF1,2019-08-29 03:45:00+03:00,2019-08-29 04:45:00+03:00,0.75
3,CONF2I,2019-08-29 04:45:00+03:00,2019-08-29 05:15:00+03:00,0.25
4,CONF1,2019-08-29 05:15:00+03:00,2019-08-29 06:30:00+03:00,1.25
5,CONF2L,2019-08-29 06:30:00+03:00,2019-08-29 07:00:00+03:00,0.50
6,CONF1,2019-08-29 07:00:00+03:00,2019-08-29 10:00:00+03:00,3.00
7,CONF2D,2019-08-29 10:00:00+03:00,2019-08-29 10:30:00+03:00,0.50
8,CONF2J,2019-08-29 10:30:00+03:00,2019-08-29 11:00:00+03:00,1.00
9,CONF3A,2019-08-29 11:00:00+03:00,2019-08-29 11:30:00+03:00,1.00


In [173]:
report_results(d, num_airspaces_in_config_dict)

3.25
3.25
0.5
4.25
1.0
5.25
0.5
6.25
1.25
7.5
0.5
8.5
3.0
11.5
0.5
12.5
0.5
13.5
0.5
15.0
1.25
17.5
1.25
20.0
1.25
22.5
0.75
23.25
1.0
25.25
2.0
27.25
0.5
28.25
1.5
29.75
0.5
30.75
50.5
81.25


{'sector_open_hours': 81.25, 'configuration_changes': 19}

In [35]:
def config_to_tfvs(df_configuration_definition, config):
    
    return df_configuration_definition[df_configuration_definition['Configuration'] == config]['TFVs'].values[0]

def get_airspace_taskloads_in_time_interval(df_sector_taskloads, df_tfv, airspace_id, 
                                            time_interval_start, time_interval_end):
    
    airspace_elementry_sectors = airspace_to_elementry_airspace_list(df_tfv, airspace_id)

    data_list = []

    taskloads = list(df_sector_taskloads[(df_sector_taskloads['time_interval_start'] == time_interval_start)
                                        &(df_sector_taskloads['time_interval_end'] == time_interval_end)
                                        &(df_sector_taskloads['elementry_sector'].isin(
                                        airspace_elementry_sectors))].iloc[:, 3:].sum())

    temp_list = [airspace_id, time_interval_start, time_interval_end] + taskloads
    data_list.append(temp_list)

    columns = ['airspace_id', 'time_interval_start', 'time_interval_end'] + list(df_sector_taskloads.columns[3:])
    out_df = pd.DataFrame(data_list, columns=columns)

    return out_df


def round_sector_configuration_df(df_sector_configuration, round_mins):
    
    out_df = df_sector_configuration.copy(deep=True)
    
    out_df['Start'] = out_df['Start'].dt.round(str(round_mins)+'min')
    out_df['End'] = out_df['End'].dt.round(str(round_mins)+'min')
    
    return out_df

def create_taskload_by_tfv_data(df_sector_taskloads, df_sector_configuration, time_interval_length_mins, 
                                df_tfv = None, sort_taskloads = False):
    
    if df_tfv is None:
        df_tfv = load_tfv_definition()
    
    max_tfvs = max([[int(e) for e in s if e.isdigit()] for s in df_sector_configuration.Configuration.to_list()])[0]

    data_list = []
    for tis in range(0,1440,time_interval_length_mins):
        tis_ts = mins_after_midnight_to_timestamp(tis)
        tie_ts = mins_after_midnight_to_timestamp(tis+time_interval_length_mins)
        config = df_sector_configuration[(df_sector_configuration['Start'] <= tis_ts)
                                        &(df_sector_configuration['End'] >= tie_ts)
                                        ]['Configuration'].values[0]
        tfv_list = config_to_tfvs(df_configuration_definition, config)
        tfv_taskloads_list = [0 for i in range(max_tfvs)] 
        for counter, tfv in enumerate(tfv_list):
            tfv_taskload = get_airspace_taskloads_in_time_interval(df_sector_taskloads, df_tfv, tfv, 
                                                                   tis_ts, tie_ts)['total_taskload'].values[0]
            tfv_taskloads_list[counter] = tfv_taskload
        # reason we sort tfv_taskloads_list is to put biggest taskload at the bottom
        if sort_taskloads:
            tfv_taskloads_list = sorted(tfv_taskloads_list,reverse=True)
        data_list.append([tis_ts, tie_ts, config, tfv_list] + tfv_taskloads_list)
    columns = ['time_interval_start', 'time_interval_end', 'config', 'tfv_list'] +\
              ['tfv ' + str(i+1) for i in range(max_tfvs)]
    df_graph = pd.DataFrame(data_list, columns = columns)
    
    return df_graph

def create_taskload_by_tfv_bar_chart(df_sector_configuration, df_graph_data):
    
    max_tfvs = max([[int(e) for e in s if e.isdigit()] for s in df_sector_configuration.Configuration.to_list()])[0]
    
    taskloads = []
    
    for counter, tfv in enumerate(list(df_graph_data.columns[-max_tfvs:])):
        hover_text = [['tfv ' + str(c+1) + ': '+ j for c, j in enumerate(i)] for i in df_graph_data.tfv_list.to_list()]
        taskloads.append(go.Bar(name=tfv, x=df_graph_data.time_interval_start.to_list(),
                                y=[i * 100 for i in df_graph_data[tfv].to_list()],
                                hovertext=hover_text
                               ))

    fig = go.Figure(data=taskloads)
    fig.update_layout(legend_orientation="h")
    fig.update_layout(barmode='stack')
    fig.update_xaxes(title='Time')
    fig.update_yaxes(title='Taskload Percentage')
    fig.show()
    
    



In [120]:
df_sector_taskloads[df_sector_taskloads['time_interval_start'] == mins_after_midnight_to_timestamp(15*44)]

,elementry_sector,time_interval_start,time_interval_end,Sequencing,Exit,Monitoring,Entry,Altitude Change,total_taskload
616,EFINA,2019-08-29 11:00:00+03:00,2019-08-29 11:15:00+03:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
617,EFINB,2019-08-29 11:00:00+03:00,2019-08-29 11:15:00+03:00,0.000000,0.005556,0.001611,0.019333,0.000000,0.026500
618,EFINC,2019-08-29 11:00:00+03:00,2019-08-29 11:15:00+03:00,0.000000,0.008333,0.010722,0.038667,0.000000,0.057722
619,EFIND,2019-08-29 11:00:00+03:00,2019-08-29 11:15:00+03:00,0.000000,0.008333,0.013935,0.077333,0.000000,0.099602
620,EFINE,2019-08-29 11:00:00+03:00,2019-08-29 11:15:00+03:00,0.022222,0.011111,0.025176,0.096667,0.038889,0.194065
621,EFINF,2019-08-29 11:00:00+03:00,2019-08-29 11:15:00+03:00,0.000000,0.013889,0.026185,0.096667,0.015556,0.152296
622,EFING,2019-08-29 11:00:00+03:00,2019-08-29 11:15:00+03:00,0.000000,0.011111,0.012861,0.077333,0.007778,0.109083
623,EFINH,2019-08-29 11:00:00+03:00,2019-08-29 11:15:00+03:00,0.000000,0.000000,0.016130,0.058000,0.007778,0.081907
624,EFINJ,2019-08-29 11:00:00+03:00,2019-08-29 11:15:00+03:00,0.000000,0.002778,0.002407,0.000000,0.007778,0.012963
625,EFINK,2019-08-29 11:00:00+03:00,2019-08-29 11:15:00+03:00,0.022222,0.002778,0.002778,0.038667,0.007778,0.074222


In [60]:
mins_after_midnight_to_timestamp(1155)

Timestamp('2019-08-29 19:15:00+0300', tz='Europe/Helsinki')